In [1]:
import requests
import pandas as pd
import csv
import re
import requests
from bs4 import BeautifulSoup
import urllib.request
import datetime

# Get League Predicitons

In [2]:
df = pd.read_csv('./football_data/spi_matches_latest.csv')
df = df.drop(
        df.index[df['date'] == 'date'])
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [3]:
df['season']

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2019,2019-03-01,1979,Chinese Super League,Shandong Luneng,Guizhou Renhe,48.22,37.83,0.5755,0.174,...,45.9,22.1,1,0,1.39,0.26,2.05,0.54,1.05,0.0
1,2019,2019-03-01,1979,Chinese Super League,Shanghai Greenland,Shanghai SIPG,39.81,60.08,0.2387,0.5203,...,25.6,63.4,0,4,0.57,2.76,0.8,1.5,0.0,3.26
2,2019,2019-03-01,1979,Chinese Super League,Guangzhou Evergrande,Tianjin Quanujian,65.59,39.99,0.7832,0.0673,...,77.1,28.8,3,0,0.49,0.45,1.05,0.75,3.15,0.0
3,2019,2019-03-01,1979,Chinese Super League,Wuhan Zall,Beijing Guoan,32.25,54.82,0.2276,0.5226,...,35.8,58.9,0,1,1.12,0.97,1.51,0.94,0.0,1.05
4,2019,2019-03-01,1979,Chinese Super League,Chongqing Lifan,Guangzhou RF,38.24,40.45,0.4403,0.2932,...,26.2,21.3,2,2,2.77,3.17,1.05,2.08,2.1,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23436,2022,2022-11-13,1859,Norwegian Tippeligaen,Kristiansund BK,FK Jerv,25.14,16.56,0.5361,0.192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23437,2022,2022-11-13,1859,Norwegian Tippeligaen,Valerenga,Molde,37.88,50.06,0.3473,0.3877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23438,2022,2022-11-13,1859,Norwegian Tippeligaen,Sandefjord,Haugesund,26.41,23.64,0.4767,0.2588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23439,2022,2022-11-13,1859,Norwegian Tippeligaen,Lillestrom,Hamarkamaratene,40.81,23.2,0.6288,0.1383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
